# Графический интерфейс Housing

In [2]:
import warnings
warnings.filterwarnings("ignore")
from PyQt5.QtWidgets import *
from PyQt5.QtCore import Qt
import sys
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

def main():
    le_fl = LabelEncoder()
    le_ro = LabelEncoder()
    le_he = LabelEncoder()
    le_fl.classes_ = np.load('floor_no_classes.npy', allow_pickle=True)
    le_ro.classes_ = np.load('room_count_classes.npy', allow_pickle=True)
    le_he.classes_ = np.load('heating_type_classes.npy', allow_pickle=True)
    model = joblib.load("best_housing_cb.pkl")
    
    outdf = pd.DataFrame(data=None, columns=["building_age", "tom", "total_floor_count",
                                    "floor_no", "room_count", "size", "heating_type", "price"])
    
    def clear_data():
        global outdf
        outdf = pd.DataFrame(data=None, columns=["building_age", "tom", "total_floor_count",
                                    "floor_no", "room_count", "size", "heating_type", "price"])
        table.setRowCount(0)
        text.clear()
        text.append('Данные удалены')
        label1.setText('0 предсказанных значений')
        label1.adjustSize()
        
    def save_output():
        fileName, _ = QFileDialog.getSaveFileName(None,"QFileDialog.getSaveFileName()","","All Files (*);;csv(*.csv)")
        outdf.to_csv(fileName,index=False, encoding='utf-8')
        text.append(f'Данные сохранены в "{fileName}"')
    
    def pred_from_user_input():
        try:
            global outdf
            outlist = [line1.text(), line2.text(), line3.text(),
                       line4.text(), line5.text(), line6.text(), line7.text()]
            X = pd.DataFrame(data=[outlist], columns=["tom", "total_floor_count", "floor_no",
                                                    "room_count", "size", "heating_type", "price"])
            
            l = np.array([line1.text(), line2.text(), *le_fl.transform(np.asarray([line3.text()])),
                          *le_ro.transform(np.asarray([line4.text()])),
                          line5.text(), le_he.transform(np.asarray([line6.text()])), line7.text()], dtype = float)
            
            #model = joblib.load("best_housing_cb.pkl")
            y_pred = model.predict(l)
            
            preddf = pd.concat([pd.DataFrame([np.round(y_pred)], columns=['building_age']), X],
                          axis=1, join='outer')
            
            outdf=outdf.append(preddf,  ignore_index=True)
                                
            # В таблицу
            table_outlist = [str(round(y_pred)), line1.text(), line2.text(), line3.text(),
                       line4.text(), line5.text(), line6.text(), line7.text()]
            table.setRowCount(table.rowCount()+1)
            for i, item in enumerate(table_outlist):
                table.setItem(table.rowCount()-1, i, QTableWidgetItem(item))
                
            # В текстовое поле
            text.append('#############################')
            text.append(f'Обработано {1} значение. (Всего {len(outdf)})')
            text.append('Данные записаны в память и выведены на экран.')
            text.append('Сохраните в файл,чтобы получить все предсказания за сеанс.')
            text.append('#############################')
            label1.setText(f'{len(outdf)} предсказанных значений')
            label1.adjustSize()
        except:
            alert = QMessageBox()
            alert.setText('Данные не введены или некорректны!')
            alert.exec_()
    
    def predict_from_file():
        try:
            global outdf
            filePath, _ = QFileDialog.getOpenFileName(None,'Open file',  "","All Files (*);;CSV Files (*.csv)")
            df = pd.read_csv(filePath)
            X = df[["tom", "total_floor_count", "floor_no", "room_count", "size", "heating_type", "price"]]
            #model = joblib.load("best_housing_cb.pkl")
            y_pred = model.predict(X)
            
            # В таблицу
            preddf = pd.concat([pd.DataFrame(np.round(y_pred), columns=['building_age']), decode_outdf(X)],
                              axis=1, join='outer')
            outdf=outdf.append(preddf,  ignore_index=True)
            table.setRowCount(table.rowCount()+20)
            for i in range(20):
                for j, column in enumerate(preddf.columns):
                    table.setItem(table.rowCount()-20+i, j, QTableWidgetItem(str(preddf.iloc[i][column])))
            
            # В текстовое поле
            text.append('#############################')
            text.append(f'Обработано {len(preddf)} значений (Всего {len(outdf)}).')
            text.append('Данные записаны в память.')
            text.append('На экран выведено первые 20 значений.')
            text.append('Сохраните в файл,чтобы получить все предсказания за сеанс.')
            text.append('#############################')
            label1.setText(f'{len(outdf)} предсказанных значений')
            label1.adjustSize()
        except:
            alert = QMessageBox()
            alert.setText('Выберите файлы "test_housing_prepr.csv" или "train_housing_prepr.csv"')
            alert.exec_()
        
    def decode_outdf(df):
        dfx = df.copy()
        dfx['floor_no'] = le_fl.inverse_transform(dfx['floor_no'])
        dfx['room_count'] = le_ro.inverse_transform(dfx['room_count'])
        dfx['heating_type'] = le_he.inverse_transform(dfx['heating_type'])
        return dfx    
        
    def display():
        line1.setText(str(slider1.value()))
        line2.setText(str(slider2.value()))
        line3.setText(str(combo3.currentText()))
        line4.setText(str(combo4.currentText()))
        line5.setText(str(slider5.value()))
        line6.setText(str(combo6.currentText()))
        line7.setText(str(slider7.value()))
        
    # Для выпадающих списков
    floor_no_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
                     '17', '18', '19', '20 ve üzeri', 'Asma Kat', 'Bahçe katı', 'Bodrum Kat', 'En Üst Kat',
                     'Giriş Katı', 'Komple', 'Kot 1', 'Kot 2', 'Kot 3', 'Kot 4', 'Müstakil', 'Teras Kat',
                     'Yüksek Giriş', 'Zemin Kat', 'nan', 'Çatı Katı']
    room_count_list = ['1+0', '1+1','2+1', '2+2', '3+1', '3+2', '4+1', '4+2', '5+1', '4+3', '5+2', '6+1', 
                       '5+3', '6+2', '7+1', '6+3', '8+1', '7+3', '8+2', '10+0', '10+1', '9+3', '9+5']
    heating_type_list = ['Fancoil', 'Güneş Enerjisi', 'Jeotermal', 'Kalorifer (Akaryakıt)',
                        'Kalorifer (Doğalgaz)', 'Kalorifer (Kömür)', 'Kat Kaloriferi',
                        'Klima', 'Kombi (Doğalgaz)', 'Kombi (Elektrikli)',
                        'Merkezi Sistem', 'Merkezi Sistem (Isı Payı Ölçer)',
                        'Soba (Doğalgaz)', 'Soba (Kömür)', 'Yerden Isıtma', 'Yok']
    
    
    app = QApplication(sys.argv) 
    w = QMainWindow()
    w.setGeometry(700,1000,1000,700)
    w.setWindowTitle("GUI")
    qr = w.frameGeometry()
    cp = QDesktopWidget().availableGeometry().center()
    qr.moveCenter(cp)
    w.move(qr.topLeft())
    
    # Ввод 'tom'
    label_min = QLabel(w)
    label_min.setText('0')
    label_min.adjustSize()
    label_min.move(40,320)
    slider1 = QSlider(Qt.Horizontal, w)
    slider1.setGeometry(50,320, 200, 50)
    slider1.setMinimum(0)
    slider1.setMaximum(250)
    slider1.setTickPosition(QSlider.TicksBelow)
    slider1.setTickInterval(10)
    slider1.valueChanged.connect(display)
    label_max = QLabel(w)
    label_max.setText('250')
    label_max.adjustSize()
    label_max.move(250,320)
    
    label = QLabel(w)
    label.setText('tom')
    label.adjustSize()
    label.move(300,300)
    
    line1 = QLineEdit(w)
    line1.move(300,320)
    
    
    # Ввод 'total_floor_count'
    label_min = QLabel(w)
    label_min.setText('1')
    label_min.adjustSize()
    label_min.move(40,370)
    slider2 = QSlider(Qt.Horizontal, w)
    slider2.setGeometry(50,370, 200, 50)
    slider2.setMinimum(1)
    slider2.setMaximum(25)
    slider2.setTickPosition(QSlider.TicksBelow)
    slider2.setTickInterval(1)
    slider2.valueChanged.connect(display)
    label_max = QLabel(w)
    label_max.setText('25')
    label_max.adjustSize()
    label_max.move(250,370)
    
    label = QLabel(w)
    label.setText('total_floor_count')
    label.adjustSize()
    label.move(300,350)
    
    line2 = QLineEdit(w)
    line2.move(300,370)
    
    
    # Ввод 'floor_no'
    combo3 = QComboBox(w)
    combo3.setGeometry(40,420, 230, 30)
    combo3.addItems(floor_no_list)
    combo3.currentTextChanged.connect(display)
    
    label = QLabel(w)
    label.setText('floor_no')
    label.adjustSize()
    label.move(300,400)
    
    line3 = QLineEdit(w)
    line3.move(300,420)
    
    
    # Ввод 'room_count'
    combo4 = QComboBox(w)
    combo4.setGeometry(40,470, 230, 30)
    combo4.addItems(room_count_list)
    combo4.currentTextChanged.connect(display)
    
    label = QLabel(w)
    label.setText('room_count')
    label.adjustSize()
    label.move(300,450)
    
    line4 = QLineEdit(w)
    line4.move(300,470)
    
    
    # Ввод 'size'
    label_min = QLabel(w)
    label_min.setText('40')
    label_min.adjustSize()
    label_min.move(35,520)
    slider5 = QSlider(Qt.Horizontal, w)
    slider5.setGeometry(50,520, 200, 50)
    slider5.setMinimum(40)
    slider5.setMaximum(250)
    slider5.setTickPosition(QSlider.TicksBelow)
    slider5.setTickInterval(10)
    slider5.valueChanged.connect(display)
    label_max = QLabel(w)
    label_max.setText('250')
    label_max.adjustSize()
    label_max.move(250,520)
    
    label = QLabel(w)
    label.setText('size')
    label.adjustSize()
    label.move(300,500)
    
    line5 = QLineEdit(w)
    line5.move(300,520)
    
    
    # Ввод 'heating_type'
    combo6 = QComboBox(w)
    combo6.setGeometry(40,570, 230, 30)
    combo6.addItems(heating_type_list)
    combo6.currentTextChanged.connect(display)
    
    label = QLabel(w)
    label.setText('heating_type')
    label.adjustSize()
    label.move(300,550)
    
    line6 = QLineEdit(w)
    line6.move(300,570)
    
    
    # Ввод 'price'
    label_min = QLabel(w)
    label_min.setText('0')
    label_min.adjustSize()
    label_min.move(40,620)
    slider7 = QSlider(Qt.Horizontal, w)
    slider7.setGeometry(50,620, 200, 50)
    slider7.setMinimum(0)
    slider7.setMaximum(800000)
    slider7.setTickPosition(QSlider.TicksBelow)
    slider7.setTickInterval(100000)
    slider7.valueChanged.connect(display)
    label_max = QLabel(w)
    label_max.setText('800k')
    label_max.adjustSize()
    label_max.move(250,620)
    
    label = QLabel(w)
    label.setText('price')
    label.adjustSize()
    label.move(300,600)
    
    line7 = QLineEdit(w)
    line7.move(300,620)
    
    # Текстовое поле
    text = QTextEdit(w)
    text.setGeometry(10,10,400,190)
    
    # Предсказание из поля ввода
    label = QLabel(w)
    label.setText('или введите данные и нажмите:')
    label.adjustSize()
    label.move(130,270)
    b1 = QPushButton(w)
    b1.setText("Predict")
    b1.move(300,260)
    b1.clicked.connect(pred_from_user_input)
    # Предсказание из файла
    label = QLabel(w)
    label.setText('Нажмите, чтобы выбрать файл для предсказания:')
    label.adjustSize()
    label.move(40,230)
    b2 = QPushButton(w)
    b2.setText("Predict file")
    b2.move(300,220)
    b2.clicked.connect(predict_from_file)
    # Сумма значений
    label1 = QLabel(w)
    label1.setText('0 предсказанных значений')
    label1.adjustSize()
    label1.move(500,570)
    label1.setAlignment(Qt.AlignRight)      
    # Очистка данных
    b3 = QPushButton(w)
    b3.setText("Clear")
    b3.move(700,560)
    b3.clicked.connect(clear_data)
    # Сохранение данных в файл
    label = QLabel(w)
    label.setText('Сохранить данные в CSV файл')
    label.adjustSize()
    label.move(500,610)
    label.setAlignment(Qt.AlignRight) 
    b4 = QPushButton(w)
    b4.setText("Save preds to csv")
    b4.move(700,600)
    b4.clicked.connect(save_output)
    
    # Таблица
    grid_layout = QGridLayout()
    w.setLayout(grid_layout)
    
    label = QLabel(w)
    label.setText('Таблица вывода:')
    label.adjustSize()
    label.move(500,10)
    
    table = QTableWidget(w)
    table.setColumnCount(8)
    table.setHorizontalHeaderLabels(["building_age (target)", "tom", "total_floor_count",
                                    "floor_no", "room_count", "size", "heating_type", "price"])
    table.resizeColumnsToContents()
    table.setGeometry(430,30, 560, 520)
    
                 
                    
    w.show()
    app.exec_()
    
if __name__ == '__main__':
    main()